# **Day 3: The Physics of Database Internals**

**Objective:** We will simulated the core components of the Oracle Architecture (SGA, WAL, Checkpoints) using Python and Linux commands. We will see how a "logical" SQL command translates into "physical" disk bytes.

**Prerequisites:**
* Linux Environment (WSL2, Mac, or Linux VM)
* Python 3.x

---

## **1.0 Memory Architecture: SGA (Shared) vs PGA (Private)**

**Concept:** 
* **SGA (System Global Area):** Memory shared by ALL processes (like the Buffer Cache).
* **PGA (Program Global Area):** Memory private to ONE process (like your Session variables).

### **Python Simulation**
We will create a "Shared Variable" (SGA) and a "Private Variable" (PGA) and spawn two processes to prove the difference.

In [ ]:
import multiprocessing
import time
import os

# Minimal function to simulate a Database User
def db_session(user_name, shared_sga):
    # PGA: This variable is PRIVATE. User B cannot see User A's PGA.
    pga_private_data = f"Session_{os.getpid()}"
    
    # SGA: This variable is SHARED. Everyone sees the update.
    shared_sga.value += 1
    
    print(f"[{user_name}] PID: {os.getpid()} | PGA: {pga_private_data} | SGA (Counter): {shared_sga.value}")

if __name__ == "__main__":
    # Create the SGA (A shared integer starting at 0)
    sga = multiprocessing.Value('i', 0)
    
    print("--- Starting Sessions ---")
    # Spawn User A
    p1 = multiprocessing.Process(target=db_session, args=("User A", sga))
    p1.start()
    p1.join()
    
    # Spawn User B
    p2 = multiprocessing.Process(target=db_session, args=("User B", sga))
    p2.start()
    p2.join()

### **Bash Equivalent (Linux Introspection)**
In Linux, we can see these Shared Memory segments using `ls` on `/dev/shm` or `ipcs`. This is literally where the SGA lives.

In [ ]:
%%bash
echo "--- Checking Shared Memory Segments (SGA) ---"
# In a real server, Oracle creates segments here
ls -l /dev/shm

echo ""
echo "--- Checking Process Memory (PGA) ---"
# We pick the current shell process to see its private memory
ps -o pid,user,vsz,rss,comm -p $$

---

## **2.0 The WAL Protocol: Redo Log vs Datafile**

**Concept:** 
Writing to the **Redo Log** (Sequential Append) is 100x faster than writing to the **Datafile** (Random Seek). This is why databases utilize a "Write-Ahead Log".

### **Python Simulation**
We will race **Sequential I/O** (Log Writer) against **Random I/O** (DB Writer).

In [ ]:
import time
import os
import random

# Setup files
redo_file = "redo.log"
data_file = "data.dbf"
data_size = 10 * 1024 * 1024 # 10MB file

# Create a dummy 10MB datafile
with open(data_file, "wb") as f:
    f.write(b'\0' * data_size)

print("--- RACING I/O (5000 Writes) ---")

# 1. Sequential Write (LGWR)
start = time.time()
with open(redo_file, "wb") as f:
    for i in range(5000):
        f.write(os.urandom(1024)) # Append 1KB
print(f"LGWR (Sequential): {time.time() - start:.4f} seconds")

# 2. Random Write (DBWn)
start = time.time()
with open(data_file, "rb+") as f:
    for i in range(5000):
        # Seek to random location
        pos = random.randint(0, data_size - 1024)
        f.seek(pos)
        f.write(os.urandom(1024)) # Write 1KB
print(f"DBWn (Random):     {time.time() - start:.4f} seconds")

### **Bash Equivalent (Raw Device Test)**
We can use the `dd` command to simulate the exact same physics at the OS level.

In [ ]:
%%bash
echo "--- Bash: Sequential Write Speed (LGWR) ---"
# Write 100MB sequentially
dd if=/dev/zero of=bash_redo.log bs=1M count=100 oflag=dsync 2>&1 | grep "copied"

echo ""
echo "--- Bash: Random Write Simulation (DBWn) ---"
# (Note: dd is mostly sequential, simulating true random seek in bash is harder,
# but we can show the difference in small block writes which hurt performance)
dd if=/dev/zero of=bash_data.dbf bs=1k count=1000 oflag=dsync 2>&1 | grep "copied"

---

## **3.0 The Commit Latency (fsync)**

**Concept:** 
A `COMMIT` is not complete until the disk says "Saved". This requires the `fsync` system call, which forces the physical drive to spin and write.

### **Python Simulation**
We compare "Writing to OS Cache" (Fast but Unsafe) vs "Forcing to Disk" (Slow but Safe).

In [ ]:
import os

print("--- Transaction Safety Test (1000 Commits) ---")

# Unsafe (No fsync) - Like insert without commit
start = time.time()
with open("unsafe.txt", "w") as f:
    for i in range(1000):
        f.write("INSERT INTO T VALUES (1);\n")
print(f"Unsafe Write: {time.time() - start:.4f} seconds (Data in RAM only)")

# Safe (With fsync) - Like COMMIT after every insert
start = time.time()
with open("safe.txt", "w") as f:
    for i in range(1000):
        f.write("INSERT INTO T VALUES (1);\n")
        f.flush()            # Push to OS
        os.fsync(f.fileno()) # Force to Disk Platter
print(f"Safe Commit:  {time.time() - start:.4f} seconds (Data on Disk)")

### **Bash Equivalent (Sync)**
We can use `sync` to manually force this flush in Linux.

In [ ]:
%%bash
echo "Writing data..."
echo "Important Data" > bash_commit.txt

# This command forces the OS to empty the Page Cache to the Disk
time sync

echo "Data synced to metal."

---

## **4.0 Physical Storage: The Block Header**

**Concept:** 
Oracle stores data in **8KB Blocks**. Each block has a **Header** (Metadata) and **Body** (Row Data).

### **Python Simulation**
Let's craft a binary file that mimics an Oracle Block.

In [ ]:
block_file = "oracle_block.bin"

# 1. Define Block Structure (Simulated)
block_id = b'\x00\x00\x01\x0A'  # Block ID: 266
table_id = b'\x00\x00\xFF\x01'  # Table ID: 65281
row_data = b'John Doe|Data Engineer|2024'

# 2. Write Binary Block
with open(block_file, "wb") as f:
    f.write(block_id)    # Header: Block Address
    f.write(table_id)    # Header: Table ID
    f.write(b'\x00' * 10)# Header: Padding/Locks
    f.write(row_data)    # Body: Actual Data
    
print(f"Created {block_file}. Now let's inspect the 'Metal' with Bash.")

### **Bash Equivalent (Hex Dump)**
A Data Engineer often needs to look at corrupt files. We use `hexdump` or `xxd` to see the raw bytes.

In [ ]:
%%bash
# Inspect the binary file we just created
# -C : Canonical hex+ASCII display
hexdump -C oracle_block.bin

**Observation:**
Look at the output above.
* The first bytes are our **Header**.
* On the right side (ASCII), you can read **"John Doe"**.
* This is exactly how `SELECT *` works: it reads raw bytes and formats them for you.

In [ ]:
# Cleanup
import os
files = ["redo.log", "data.dbf", "unsafe.txt", "safe.txt", "oracle_block.bin", "bash_redo.log", "bash_data.dbf", "bash_commit.txt"]
for f in files:
    if os.path.exists(f):
        os.remove(f)
print("Cleanup Complete.")